# Band gap prediction of inorganic materials

In [121]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

## Data Preprocessing

#### Load the dataset

This dataset provides quantitative measurements of the band gap (Egap) for a set of inorganic crystaline materials.
- train_X.csv: Material and descriptors with Id column, for training
- train_y.csv: Egap with Id column, for training
- test_X.csv: Material and descriptors with Id column, for prediction

Data fields:
- Id : a numeric id unique to a given material (the last column)
- D1-D132 : chemical descriptors for a given material
- Egap : Band gap values

In [3]:
#training set
X = pd.read_csv("train_X.csv")
Y = pd.read_csv("train_y.csv")
#test set
X_test = pd.read_csv("test_X.csv")
#store the ID
ID = X_test["Id"]

print(X.shape)
print(Y.shape)
print(X_test.shape)


#### Remove the missing values

Check if there are any missing values or columns that only contain zeros values.

In [ ]:
print("missing values: ",X.isnull().sum().max())
list_zeros_X = (X == 0).sum()
list_zeros_test = (X_test == 0).sum()
num_rows_X = len(X)
num_rows_test = len(X_test)
columns_all_zeros_X = list_zeros_X[list_zeros_X == num_rows_X].index.tolist()
columns_all_zeros_test = list_zeros_test[list_zeros_test == num_rows_test].index.tolist()

print("Columns that contain only zeros in X:", columns_all_zeros_X)
print("Columns that contain only zeros in X_test:", columns_all_zeros_test)


missing values:  0
Columns that contain only zeros in X: ['D97', 'D121']
Columns that contain only zeros in X_test: ['D97', 'D121']


In [ ]:
## Drop the D97 and D121 column since they only contain 0 values
X.drop(['D97','D121'], axis=1, inplace=True)
X_test.drop(['D97','D121'], axis=1, inplace=True)
X.head(2)

,Material,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D124,D125,D126,D127,D128,D129,D130,D131,D132,Id
0,O4V1Y1,8,39,31,15.6667,10.2222,8,12,87,75,...,0.0,0.0,0.0,12,229,217,78.500,88.6667,12,8683
1,Al1Ba2Cu2F11,9,56,47,17.6250,12.4375,9,9,93,84,...,0.0,0.0,0.0,15,229,214,81.125,90.9219,15,8788


### Deal with the Duplicated rows

In [ ]:
def check_identical_rows_except_material(group):
    # drop the 'Material' column to compare the rest of the columns
    group_without_material = group.drop(columns='Material')
    # Compare all rows to the first row to check for identical rows
    return group_without_material.eq(group_without_material.iloc[0]).all(axis=None)

# Apply the function to each group
identical_rows_result_train = X.groupby('Material').apply(check_identical_rows_except_material)
print("There are duplicated rows in X_train:", identical_rows_result_train.any())
identical_rows_result_test = X_test.groupby('Material').apply(check_identical_rows_except_material)
print("There are duplicated rows in X_test:", identical_rows_result_test.any())

#"true" means all rows are identical across all columns except for the 'Material' column itself.

There are duplicated rows in X_train: True
There are duplicated rows in X_test: True


In [ ]:
# merge the tables
combined_df = pd.merge(X, Y, on='Id')
print(combined_df.shape)
combined_df.head(5)

(9640, 133)


,Material,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D125,D126,D127,D128,D129,D130,D131,D132,Id,Egap
0,O4V1Y1,8,39,31,15.6667,10.22220,8,12,87,75,...,0.0,0.0,12,229,217,78.500,88.6667,12,8683,2.8121
1,Al1Ba2Cu2F11,9,56,47,17.6250,12.43750,9,9,93,84,...,0.0,0.0,15,229,214,81.125,90.9219,15,8788,2.5128
2,Li2O2Pd1,3,46,43,13.6000,12.96000,3,1,87,86,...,0.0,0.0,12,229,217,141.400,103.5200,12,5144,1.9510
3,Br2Cl2Cu1Rb2,17,37,20,29.5714,7.34694,17,4,95,91,...,0.0,0.0,64,229,165,134.143,80.1633,64,9593,1.0099
4,Al1K1O2,8,19,11,12.0000,4.00000,8,3,87,84,...,0.0,0.0,12,229,217,119.500,107.5000,12,2027,2.9344


In [ ]:
df_X_without_id = X.drop(columns=['Id'])
grouped_df = combined_df.groupby(list(df_X_without_id.columns)).agg({'Egap': 'mean'}).reset_index()
print(grouped_df.shape)
grouped_df.head(5)

(8462, 132)


,Material,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D124,D125,D126,D127,D128,D129,D130,D131,D132,Egap
0,Ac1Br3,35,89,54,48.5,20.25,35,14,95,81,...,0.0,0.0,0.0,64,225,161,104.250,60.375,64,4.1084
1,Ac1Cl3,17,89,72,35.0,27.00,17,14,94,80,...,0.0,0.0,0.0,64,225,161,104.250,60.375,64,5.0778
2,Ac2O3,8,89,81,40.4,38.88,8,14,87,73,...,0.0,0.0,0.0,12,225,213,97.200,102.240,12,3.5216
3,Ag10C2F8,6,47,41,27.7,19.30,47,65,93,28,...,0.0,0.0,0.0,15,225,210,137.900,98.320,225,2.2565
4,Ag18O21Si6,8,47,39,24.4,18.08,8,65,87,22,...,0.0,0.0,0.0,12,227,215,125.867,106.276,12,0.7077


In [ ]:
#split into X and Y again
X_new = grouped_df.drop(columns=['Egap'])
Y_new = grouped_df[['Egap']]
print(X_new.shape)
print(Y_new.shape)

(8462, 131)
(8462, 1)


In [ ]:
X_new.drop(['Material'], axis=1, inplace=True) #drop material
X_test.drop(['Id'], axis=1, inplace=True) #drop ID
X_test.drop(['Material'], axis=1, inplace=True) #drop material

Y_new = Y_new.values.ravel()

## Data Scaling and Spliting

In [ ]:
#scale the data
scalar = StandardScaler()
X_train_scale = scalar.fit_transform(X_new)
X_test_scale = scalar.transform(X_test)

In [ ]:
## Split into training and validation set
train_x, validation_x, train_y, validation_y = \
    train_test_split(X_train_scale, Y_new, test_size=.20, random_state=66)

## Regression Model

### Pipeline

In [ ]:
def Pipeline(model, train_X, train_y, test_X, test_y=None, grid_run=0, grid_param={}):
    if grid_run == 1:
        model = GridSearchCV(estimator=model,
                            param_grid=grid_param,
                            cv=5, verbose = 1,
                            n_jobs=-1, scoring="neg_mean_absolute_error")
    model.fit(train_X, train_y)
    pred_y = model.predict(test_X)
    mae = 0.0
    if test_y is not None:
        mae = metrics.mean_absolute_error(test_y , pred_y)
    if grid_run == 1:
        print("Tuned hyperparameters :(best parameters) ",model.best_params_)
        model = model.best_estimator_

    print(type(model).__name__)
    print(f"Mean Absolute Error: {mae}\n")

    return model, mae

### Default models comparisons

In [ ]:
## Default Models
# Linear, Ridge, Lasso Regression
_, mae_lin = Pipeline(LinearRegression(), train_x, train_y, validation_x, validation_y, 0, {})
_, mae_ridge = Pipeline(Ridge(), train_x, train_y, validation_x, validation_y, 0, {})
_, mae_lasso = Pipeline(Lasso(), train_x, train_y, validation_x, validation_y, 0, {})

# SVR
_, mae_svr = Pipeline(SVR(), train_x, train_y, validation_x, validation_y, 0, {})

# Random forest
_, mae_rf = Pipeline(RandomForestRegressor(), train_x, train_y, validation_x, validation_y, 0, {})

# XGBoost
_, mae_xgb = Pipeline(XGBRegressor(), train_x, train_y, validation_x, validation_y, 0, {})

# GBM
_, mae_gbm = Pipeline(GradientBoostingRegressor(), train_x, train_y, validation_x, validation_y, 0, {})

LinearRegression
Mean Absolute Error: 0.8130804096823774

Ridge
Mean Absolute Error: 0.8127883843017288

Lasso
Mean Absolute Error: 1.2570136559786744

SVR
Mean Absolute Error: 0.6541146837048283

RandomForestRegressor
Mean Absolute Error: 0.5951908325259494

XGBRegressor
Mean Absolute Error: 0.5821311576811616

GradientBoostingRegressor
Mean Absolute Error: 0.6861992563867895



## Fine-tuning models with Grid Search

#### SVR

In [ ]:
# SVR with grid search
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'epsilon': [0.01, 0.1, 1],  # Epsilon in the epsilon-SVR model
}

best_svr_model, mae_svr_grid = Pipeline(SVR(kernel='rbf'), train_x, train_y, validation_x, validation_y, 1, param_grid)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Tuned hyperparameters :(best parameters)  {'C': 10, 'epsilon': 0.1}
SVR
Mean Absolute Error: 0.6056060636393861



#### Random Forest

In [ ]:
param_grid = {
    'n_estimators': [200, 300, 400, 500],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'max_features': ["sqrt","log2"]
}

best_rf_model, mae_rf_grid = Pipeline(RandomForestRegressor(random_state=42), train_x, train_y, validation_x, validation_y, 1, param_grid)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Tuned hyperparameters :(best parameters)  {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 500}
RandomForestRegressor
Mean Absolute Error: 0.5929114571882979



#### GBM

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    #'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

best_gbm_model, mae_gbm_grid = Pipeline(GradientBoostingRegressor(random_state=42), train_x, train_y, validation_x, validation_y, 1, param_grid)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Tuned hyperparameters :(best parameters)  {'learning_rate': 0.2, 'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 300}
GradientBoostingRegressor
Mean Absolute Error: 0.5835331555951866



#### XGBoost

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'colsample_bytree': [0.3, 0.7]
}

best_xgb_model, mae_xgb_grid = Pipeline(XGBRegressor(objective="reg:squarederror",n_estimators = 5000,eval_metric="mae"), train_x, train_y, validation_x, validation_y, 1, param_grid)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Tuned hyperparameters :(best parameters)  {'colsample_bytree': 0.3, 'learning_rate': 0.05, 'max_depth': 5}
XGBRegressor
Mean Absolute Error: 0.5277638239835968



### Stacking

In [ ]:
level0 = list()
level0.append(('rf', best_rf_model))
level0.append(('xgb', best_xgb_model))
level0.append(('svr', best_svr_model))

# Meta learner model
level1 = best_svr_model

model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

model.fit(train_x, train_y)

# Make predictions
prediction = model.predict(validation_x)
mae_stack = metrics.mean_absolute_error(prediction, validation_y)
print("Mean Absolute Error: ", mae_stack)

Mean Absolute Error:  0.5253086998161718


In [ ]:
model

StackingRegressor(cv=5,
                  estimators=[('rf',
                               RandomForestRegressor(max_features='sqrt',
                                                     n_estimators=500,
                                                     random_state=42)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.3, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='mae',
                                            feature_types=None, g...
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                              ('svr', SVR(C=10))],
                  final_estimator=SVR(C=10))

### Blending

In [ ]:
train_x_base, train_x_meta, train_y_base, train_y_meta = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
rf_model = best_rf_model.fit(train_x_base, train_y_base)
xgb_model = best_xgb_model.fit(train_x_base, train_y_base)
svr_model = best_svr_model.fit(train_x_base, train_y_base)

# Predict on holdout set
rf_pred = rf_model.predict(train_x_meta)
xgb_pred = xgb_model.predict(train_x_meta)
svr_pred = svr_model.predict(train_x_meta)

# Create new feature set for the blender model
blended_features = np.column_stack((rf_pred, xgb_pred, svr_pred))

# Train the blender model
blender = SVR().fit(blended_features, train_y_meta)

# Predict on test set
rf_test_pred = rf_model.predict(validation_x)
xgb_test_pred = xgb_model.predict(validation_x)
svr_test_pred = svr_model.predict(validation_x)

# Create blended test features
blended_test_features = np.column_stack((rf_test_pred, xgb_test_pred, svr_test_pred))

# Make final predictions
final_predictions = blender.predict(blended_test_features)


mae_blend = metrics.mean_absolute_error(final_predictions , validation_y)
print("Mean Absolute Error: ", mae_blend)

Mean Absolute Error:  0.5556714952522378


## Output Y prediction into CSV

The XGBoost and Stacking model are the top 2 models that has lowest MAE scores.

In [ ]:
def output_file(model):
  model.fit(X_train_scale,Y_new)
  Y_test = model.predict(X_test_scale)
  data = {'Id': ID,
        'Egap': Y_test}
  df = pd.DataFrame(data)
  #df.to_csv('out.csv',index=False)
  return df

In [ ]:
df_xgb = output_file(best_xgb_model)
df_xgb.to_csv('xgboost.csv',index=False)

df_stack = output_file(model)
df_stack.to_csv('stack_model.csv',index=False)